In [15]:
from func import *
from typing import Callable, Iterable, List, Tuple
import plotly.graph_objects as go

In [16]:
# fetch the dataset
haxby_dataset = datasets.fetch_haxby(subjects= [1,2,3,4,5,6])


In [17]:
# Creating stimuli to category and category to stimuli:
stimuli2category = {
                        'scissors'     : 0,
                        'face'         : 1, 
                        'cat'          : 2,
                        'scrambledpix' : 3,
                        'bottle'       : 4,
                        'chair'        : 5,
                        'shoe'         : 6,
                        'house'        : 7
}

category2stimuli = {category:stimuli for stimuli, category in stimuli2category.items()}

def fetch_haxby_per_subject(subject_id:int = None,standardize:bool = True) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    
        Given the subject id, fetch the haxby data in matrix format.
        
        Arguments:
            - subject_id  (int) : Subject number from [1,6]
            - standardize (bool): If true, masks are standardized
            
        Returns:
            - data (Tuple[np.ndarray, np.ndarray, np.ndarray]) = Original 4-D data, Flattened + Masked Data, Label  
    
    """
        
    # Getting the data file name:
    spatio_temporal_data_path = haxby_dataset.func[subject_id]  
   
    # Getting labels:
    behavioral = pd.read_csv(haxby_dataset.session_target[subject_id], delimiter = ' ')
    
    # Creating conditional categories:
    conditions = behavioral['labels']
    
    # Creating masks for stimuli categories, (ignores rest conditions)
    condition_mask = conditions.isin([*stimuli2category]).tolist()
    
    # Appylying masks to labels (categorical):
    conditions = conditions[condition_mask]
    
    # Creating labels series (numerical):
    categories = np.array([stimuli2category[stimulus] for stimulus in conditions])
    
    # Masking fMRI images: (shape = (40, 64, 64, 864))
    fmri_niimgs = index_img(spatio_temporal_data_path, condition_mask)
    
    # Converting NumPy and transposing to (864, 40, 64, 64):
    numpy_fmri = fmri_niimgs.get_fdata().transpose(3,0,1,2)
    
    masker = NiftiMasker(mask_img=haxby_dataset.mask_vt[subject_id],
                         smoothing_fwhm=4,
                         standardize=standardize,
                         memory='nilearn_cache',
                         memory_level=1)

    masked = masker.fit_transform(fmri_niimgs)
    
    return numpy_fmri,  masked, categories

We mask regions of interest to the original 4D fMRI data and extract it into a numpy matrix. 

In [18]:
data = [fetch_haxby_per_subject(subject_id) for subject_id in range(6)]
fmri_imgs_mat, masks, categories = list(zip(*data))

In [19]:
# set the dir to save plots
explanatory_fMRI_dir = "images"

In [20]:
from visualizer.plot2D import plot_2d
from visualizer.plot3D import plot_3d 

In [21]:
from utils.timers import timeit
from utils.metrics import accuracy, confusion_matrix, visualize_confusion_matrix
from utils.savers import save, save_obj, load, load_obj
from utils.reproduce import random_seed
from dataset.fetch_data_matrix import fetch_from_haxby


### Unsupervised & Manifold Learning in Human Brain

Functional MRI data are very high-dimensional if one considers all the voxels or surface coordinates acquired with standard imaging parameters. As in our dataset, with the structure of 4D time-series image data, we have a curve of dimensionality problem. In fMRI data, each data point represents an image at a specific point in time, so the dimensionality of the data is determined by the number of voxels (3D coordinates) in each image and the number of time points. Therefore, as we increase the number of voxels in each image or the numberof time points, the amount of data required to represent the entire dataset grows exponentially.


The need for **dimension reduction** and **manifold learning** often arises when very high-dimensional data
needs to be analyzed but the intrinsic dimensionality of the data is much lower. This situations occurs, for example, when trying to visualize variability and common patterns in a given group of functional magnetic resonance images (fMRIs) of the brain. Each image can be regarded as a point in a high-dimensional image space (called the __ambient space__), with $n_t \times n_x \times n_y \times n_z$ coordinates, where $n_x, n_y, n_z$ are the dimensions of an image slice and $n_t$ is the temperoal dimension. On the other hand, each image could also be identified by a smaller set of parameters that describe shape variations and patterns that are common for a particular group of images. These parameters span a new space called the manifold space. The task of manifold learning is to discover the low-dimensional space and its parameters which can then be used to model the anatomical variability within a population.

Various methods have been proposed to extract features from fMRIs. Early developments in manifold learning focused on linear techniques such as principal component analysis (PCA) and multidimensional scaling (MDS). To capture the nonlinear relationships in the data, Isomap, locally linear embedding (LLE), and Laplacian eigenmaps are proposed in the late 1990s. These techniques aimed to capture the nonlinear structure of the data by preserving the local neighborhood relationships between data points. Isomap, for example, used a geodesic distance metric to preserve the global geometry of the data, while LLE focused on preserving the local geometry of the data. Isomaps and LEM are the most popular methods for medical image analysis and both of them require a prebuild proximity graph. In order to build the proximity graph, it is assumed that the manifold space is locally linear, which means that distances between neighboring points in manifold space can be approximated by their distances in ambient space.


In this section, we apply some `dimension reduction and manifold learning` methods to our 4D fMRIs dataset and visualize them in 3D. Hopefully, we can discover some interesting patterns to help us with brain decoding. 

<table>
  <tr>
    <td><img src="images/mf_1.png" width="600" height="400"></td>
    <td><img src="images/mf_2.png" width="800" height="400"></td>
  </tr>
</table>

We set all categorical labels to integers as follows:
**'scissors'**     : 0,
**'face'**         : 1, 
**'cat'**          : 2,
**'scrambledpix'** : 3,
**'bottle'**       : 4,
**'chair'**        : 5,
**'shoe'**         : 6,
**'house'**        : 7
, and we extract 864 time points when the subject is given images of above categories.

In [22]:
subject_id = 0
x = masks[subject_id]
print(x.shape)
y = categories[subject_id]
print(y.shape)

(864, 577)
(864,)


x stores the embedded values of 577 points of interest in a total time points of 864.

## Dimension Reduction Methods

| Column 1 | Column 2 | 
|----------|----------|
| ![Image 1](images/dc/pca_2d.png) | ![Image 2](images/dc/pca_3d.png) | 
| ![Image 3](images/dc/lda_3d.png) | ![Image 4](images/dc/ICA_3d.png) | 

### Principal Component Analysis (PCA)

PCA is a popular linear unsupervised dimension reduction algorithm. It finds the principal components of the data, which are the orthogonal linear combinations of the original features that explain the maximum amount of variance in the data. PCA then projects the data onto a lower-dimensional space defined by the principal components, resulting in a lower-dimensional representation of the data.

Here, we apply the PCA to RoI’s of subject 1 and visualize 2D and 3D result.

In [23]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x)

principal = pd.DataFrame(data = principalComponents
             ,columns = ['principal component 1',
                         'principal component 2',
                         'principal component 3'])

plot_2d(principalComponents[:, 0],
        principalComponents[:, 1],
        y = categories[subject_id],
        path = 'images/pca_2d.png',
        title="2D PCA Plot"
       )

In [24]:
plot_3d(principalComponents[:, 0],
        principalComponents[:, 1],
        principalComponents[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'pca_3d.png'),
        y = categories[subject_id],
        title="3D PCA Plot")

### Linear Discriminant Analysis (LDA)

LDA is supervised dimensionality reduction algorithm and it is a generalization of Fisher’s linear discriminant, aims to find linear subspace that characterize the original data space. Since it is supervised, it is a powerful paradigm
in representation learning.

In [25]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

X_LDA = LDA(n_components=3).fit_transform(x,y)

plot_3d(X_LDA[:, 0],
        X_LDA[:, 1],
        X_LDA[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'lda_3d.png'),
        y = categories[subject_id],
        title="3D LDA Plot")

In [26]:
plot_2d(X_LDA[:, 0],
        X_LDA[:, 1],
        path = os.path.join(explanatory_fMRI_dir, 'lda_3d.png'),
        y = categories[subject_id],
        title="2D LDA Plot")

### Independent Component Analysis (ICA)

Independent Component Analysis is a statistical technique for separating a set of observed signals into independent, non-Gaussian components by assuming that the observed signals are linear mixtures of the underlying independent components.

In [27]:
from sklearn.decomposition import FastICA
fast_ica = FastICA(n_components = 3)
ICs = fast_ica.fit_transform(x)

plot_3d(ICs[:, 0],
        ICs[:, 1],
        ICs[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'ica_3d.png'),
        y = categories[subject_id],
        title = "3D ICA Plot")

## Manifold Learning



Here, we will introduce some basic indeas of common manifold methods. Most manifold learning algorithms assume that **the input data resides on or close to a low-dimensional manifold embedded in the ambient space**. Take a closer look to the the algorithm of isometric embedding, we can see how it preserves geodesic distance in the high dimension.

1. **Construct a neighborhood graph**: Given a high-dimensional data set, we first construct a neighborhood graph by connecting each data point to its k nearest neighbors in the high-dimensional space.
2. **Calculate shortest path distances**: We then calculate the shortest path (geodesic) distances between all pairs of connected points in the neighborhood graph.
3. **Compute pairwise distances in the low-dimensional space by euclidean distance metric**
4. **Find the optimal low-dimensional embedding**: Finally, we find the optimal low-dimensional embedding that preserves the pairwise distances computed in step 2 as closely as possible. 

<tr>
    <td><img src="images/isomap_explain.png"></td>
</tr>

<tr>
    <td><img src="images/mf_explain.png" width="600" height="400"></td>
</tr>

| Column 1 | Column 2 | Column 3 |
|----------|----------|----------|
| ![Image 1](images/ml/isomap_3d.png) | ![Image 2](images/ml/lle_3d.png) | ![Image 3](images/ml/mds_3d.png) |
| ![Image 4](images/ml/UMAP_3d.png) | ![Image 5](images/ml/t-sne_3d.png) | ![Image 6](images/ml/spectral_embedding_3d.png) |


### Uniform Manifold Approximation and Projection (UMAP)

UMAP uses a probabilistic framework that optimizes a low-dimensional embedding to approximate the high-dimensional data, and it emphasizes the preservation of local neighborhoods and connectivity.UMAP is particularly useful for visualizing high-dimensional data in a way that facilitates pattern recognition and data exploration, and it has become a popular alternative to t-SNE and PCA for unsupervised dimensionality reduction.

In [28]:
import umap.umap_ as umap

reducer = umap.UMAP(n_components=3, n_epochs=350)
embedding = reducer.fit_transform(x)


plot_3d(embedding[:, 0],
        embedding[:, 1],
        embedding[:, 2],
         path = os.path.join(explanatory_fMRI_dir, 'umap_3d.png'),
        y = categories[subject_id],
        title="3D UMAP Plot")

### Mutidimensional Scaling (MDS)

MDS seeks a low-dimensional representation of the data in which the distances respect well the distances in the original high-dimensional space. In general, MDS is a technique used for analyzing similarity or dissimilarity data. It attempts to model similarity or dissimilarity data as distances in a geometric spaces. The data can be ratings of similarity between objects, interaction frequencies of molecules, or trade indices between countries.

In [29]:
from sklearn.manifold import MDS


embedding = MDS(n_components=3)
manifold = embedding.fit_transform(x,categories[subject_id])


plot_3d(manifold[:, 0],
        manifold[:, 1],
        manifold[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'mds_3d.png'),
        y = categories[subject_id],
        title="3D MDS Plot")

### ISOMAP

Isometric Mapping is one of the earliest non-linear manifold learning algorithm, and it seeks a lower-dimensional embedding which maintains geodesic distances between all points. This ensures that the structure of the data in the lower-dimensional space is as close as possible to the structure of the data in the original high-dimensional space.

In [30]:
from sklearn.manifold import Isomap
x = masks[subject_id]

embedding = Isomap(n_components=3, n_neighbors= 5)
manifold = embedding.fit_transform(x)


plot_3d(manifold[:, 0],
        manifold[:, 1],
        manifold[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'isomap_3d.png'),
        y = categories[subject_id],
        title="3D ISOMAP Plot")

In [31]:
plot_2d(manifold[:, 0],
        manifold[:, 1],
        path = os.path.join(explanatory_fMRI_dir, 'isomap_3d.png'),
        y = categories[subject_id],
        title="2D ISOMAP Plot")

### Locally Linear Embedding

Locally linear embedding (LLE) seeks a lower-dimensional projection of the data which preserves distances within local neighborhoods. It can be thought of as a series of local Principal Component Analyses which are globally compared to find the best non-linear embedding.

In [32]:
from sklearn.manifold import LocallyLinearEmbedding


embedding = LocallyLinearEmbedding(n_components=3)
manifold = embedding.fit_transform(x,categories[subject_id])


plot_3d(manifold[:, 0],
        manifold[:, 1],
        manifold[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'lle_3d.png'),
        y = categories[subject_id],
        title="3D LLE Plot")

### Spectral Embedding
Spectral embedding is also non-linear embedding algorithm that forms an affinity matrix and applies spectral decomposition to the laplacian graph.
This technique relies on the basic assumption that the data lies in a low-dimensional manifold in a high-dimensional space.The Spectral Embedding consists of three stages:
1. Weighted Graph Construction

    Transform the raw input data into graph representation using affinity (adjacency) matrix representation. 

2. Graph Laplacian Construction: $L = D^{-\frac{1}{2}}(D-A)D^{-\frac{1}{2}}$
3. Partial Eigenvalue Decomposition: Eigenvalue decomposition is done on graph Laplacian

In [33]:
from sklearn.manifold import SpectralEmbedding

embedding = SpectralEmbedding(n_components=3)
manifold = embedding.fit_transform(x)

plot_3d(manifold[:, 0],
        manifold[:, 1],
        manifold[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'SpectralEmbedding_3d.png'),
        y = categories[subject_id],
        title="3D Spectral Embedding Plot")

In [34]:
plot_2d(manifold[:, 0],
        manifold[:, 1],
        path = os.path.join(explanatory_fMRI_dir, 'SpectralEmbedding_3d.png'),
        y = categories[subject_id],
        title="2D Spectral Embedding Plot")

### T-Stochastic Neighbor Embedding (t-SNE)

t-SNE converts affinities of data points to probabilities. The affinities in the original space are represented by Gaussian joint probabilities and the affinities in the embedded space are represented by Student’s t-distributions. It minimizes the Kullback-Leibler divergence between the joint probablities of the low-dimensional embedding and the high-dimensional data. t-SNE focuses on the local structure of the data and will tend to extract clustered local groups of samples.

In [35]:
from sklearn.manifold import TSNE
x = masks[subject_id]

tsne = TSNE(random_state = 42,
            n_components=3,
            verbose=0,
            perplexity=40,
            n_iter=400).fit_transform(x)

In [36]:
plot_2d(tsne[:, 0],
        tsne[:, 1],
        path = os.path.join(explanatory_fMRI_dir, 'tsene_2d.png'),
        y = categories[subject_id],
        title="2D t-SNE Plot")

In [37]:
plot_3d(tsne[:, 0],
        tsne[:, 1],
        tsne[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'tsene_3d.png'),
        y = categories[subject_id],
        title="3D t-SNE Plot")

### Results

- All methods except for LDA demonstrate that there are highly overlapping and distributed representations in that areas. 
- LDA show that even there are highly correlated and overlapping responses, we can discriminate them by powerful decoding algorithm. 
- Some manifold learning methods utilize deep learning show the great fMRI decoing ability, but we do not have time to present them in this report.

### Reference:

“Laplacian Eigenmaps for Dimensionality Reduction and Data Representation” M. Belkin, P. Niyogi, Neural Computation, June 2003; 15 (6):1373-1396